In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join
from lxml import etree

from PIL import Image
from PIL import ImageOps as IO
# import cv2

In [4]:
# Define data directories
my_data = "../data"
annotations = my_data + '/Annotation'        # BO file to be merged with additional information
images = my_data + '/Images' # SAP file with order fulfilment dates
cropped = my_data + '/Cropped'

In [3]:
# Convert all pictures into cropped and autocontrasted version
# Explore all picture folders
folders = [f for f in listdir(annotations) if (~isfile(join(annotations, f)) &
                                                f.lower().startswith('n02'))]

for folder in folders:
    print(folder)
    current_folder = join(annotations, folder)
    files = [f for f in listdir(current_folder) if (isfile(join(current_folder, f)))]
    for file in files:
        # Read each picture
        current_file = join(current_folder, file)      
        picture_file = join(images, folder, file + '.jpg')
        img = Image.open(picture_file) 
        
        # Get crop region for each picture
        tree = etree.parse(current_file)
        for coord in tree.xpath("/annotation/object/bndbox"):
            xmin_el = int(coord.xpath("xmin")[0].text)
            xmax_el = int(coord.xpath("xmax")[0].text)
            ymin_el = int(coord.xpath("ymin")[0].text)
            ymax_el = int(coord.xpath("ymax")[0].text)
        
        img_cropped = img.crop((xmin_el,ymin_el,xmax_el,ymax_el))
        try:
            img_cropped = IO.grayscale(img_cropped)
            img_cropped = IO.equalize(img_cropped)
            # img_cropped = IO.autocontrast(img_cropped, 1)
        except 'OSError':
            pass
       
        out_picture_file = join(cropped, folder, file + '.jpg')
        img_cropped.save(out_picture_file)


n02085620-Chihuahua
n02085782-Japanese_spaniel
n02085936-Maltese_dog
n02086079-Pekinese
n02086240-Shih-Tzu
n02086646-Blenheim_spaniel
n02086910-papillon
n02087046-toy_terrier
n02087394-Rhodesian_ridgeback
n02088094-Afghan_hound
n02088238-basset
n02088364-beagle
n02088466-bloodhound
n02088632-bluetick
n02089078-black-and-tan_coonhound
n02089867-Walker_hound
n02089973-English_foxhound
n02090379-redbone
n02090622-borzoi
n02090721-Irish_wolfhound
n02091032-Italian_greyhound
n02091134-whippet
n02091244-Ibizan_hound
n02091467-Norwegian_elkhound
n02091635-otterhound
n02091831-Saluki
n02092002-Scottish_deerhound
n02092339-Weimaraner
n02093256-Staffordshire_bullterrier
n02093428-American_Staffordshire_terrier
n02093647-Bedlington_terrier
n02093754-Border_terrier
n02093859-Kerry_blue_terrier
n02093991-Irish_terrier
n02094114-Norfolk_terrier
n02094258-Norwich_terrier
n02094433-Yorkshire_terrier
n02095314-wire-haired_fox_terrier
n02095570-Lakeland_terrier
n02095889-Sealyham_terrier
n02096051-Aired

In [40]:
# List number of pictures per folder

folders = [f for f in listdir(annotations) if (~isfile(join(annotations, f)) &
                                                f.lower().startswith('n02'))]

for folder in folders:
    current_folder = join(annotations, folder)
    files = [f for f in listdir(current_folder) if (isfile(join(current_folder, f)))]
    print("{0} images for {1}".format(len(files), folder))
    

152 images for n02085620-Chihuahua
185 images for n02085782-Japanese_spaniel
252 images for n02085936-Maltese_dog
149 images for n02086079-Pekinese
214 images for n02086240-Shih-Tzu
188 images for n02086646-Blenheim_spaniel
196 images for n02086910-papillon
172 images for n02087046-toy_terrier
172 images for n02087394-Rhodesian_ridgeback
239 images for n02088094-Afghan_hound
175 images for n02088238-basset
195 images for n02088364-beagle
187 images for n02088466-bloodhound
171 images for n02088632-bluetick
159 images for n02089078-black-and-tan_coonhound
153 images for n02089867-Walker_hound
157 images for n02089973-English_foxhound
148 images for n02090379-redbone
151 images for n02090622-borzoi
218 images for n02090721-Irish_wolfhound
182 images for n02091032-Italian_greyhound
187 images for n02091134-whippet
188 images for n02091244-Ibizan_hound
196 images for n02091467-Norwegian_elkhound
151 images for n02091635-otterhound
200 images for n02091831-Saluki
232 images for n02092002-Sc

In [8]:
def gen_sift_features(gray_img):
    sift = cv2.xfeatures2d.SIFT_create()
    # kp is the keypoints
    #
    # desc is the SIFT descriptors, they're 128-dimensional vectors
    # that we can use for our final features
    kp, desc = sift.detectAndCompute(gray_img, None)
    return kp, desc

def show_sift_features(gray_img, color_img, kp):
    return plt.imshow(cv2.drawKeypoints(gray_img, kp, color_img.copy()))

def load_img(img_folder, img_file):
    picture_file = join(cropped, img_folder, img_file)
    img = Image.open(picture_file)
    img_sift = np.array(img)
    return img_sift

folder = "n02085782-Japanese_spaniel"
dog_file = "n02085782_23.jpg"
# generate SIFT keypoints and descriptors
dog1_img = load_img(folder, dog_file)
dog1_kp, dog1_desc = gen_sift_features(dog1_img)

dog_file = "n02085782_3030.jpg"
dog2_img = load_img(folder, dog_file)
dog2_kp, dog2_desc = gen_sift_features(dog2_img)

print 'SIFT Features:'
show_sift_features(dog1_img, dog1_desc, dog1_kp)



SIFT Features:


In [9]:
len(dog1_desc[0])


128

In [10]:
from sklearn.cluster import KMeans
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(dog1_desc)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [11]:
clusters = kmeans.labels_

In [12]:
len(clusters)

317

In [13]:
img_histogram = np.histogram(clusters, bins=n_clusters, range=(0,n_clusters))
norm_histogram = np.divide(img_histogram, np.array([len(clusters), 1.0]))
norm_histogram

array([ array([ 0.07255521,  0.09148265,  0.13564669,  0.10094637,  0.10094637,
        0.07570978,  0.08201893,  0.10410095,  0.08832808,  0.14826498]),
       array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])], dtype=object)

In [15]:
np.divide(img_histogram, np.array([27, 1.0]))

array([ array([ 0.85185185,  1.07407407,  1.59259259,  1.18518519,  1.18518519,
        0.88888889,  0.96296296,  1.22222222,  1.03703704,  1.74074074]),
       array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])], dtype=object)

In [16]:
kmeans.cluster_centers_

array([[ 26.39130592,  27.17391205,  21.04347801, ...,  17.86956406,
         28.47826004,  19.60869598],
       [ 14.17241383,   9.00000477,   5.10344315, ...,  15.44827461,
          3.62069178,   5.37931061],
       [ 12.37208843,  22.93023491,  26.65116501, ...,  25.62790489,
         22.79069519,  17.1860466 ],
       ..., 
       [ 20.42424393,  10.00000381,  10.96969604, ...,   8.30302906,
         10.39393902,  16.39393997],
       [ 33.6071434 ,  41.28572083,  33.99999619, ...,   9.        ,
         12.21428394,  24.7142868 ],
       [ 19.72340584,   9.68085575,   8.12766361, ...,   3.85106659,
          5.1914897 ,  11.95744705]], dtype=float32)

In [17]:
show_sift_features(dog2_img, dog2_desc, dog2_kp)

In [18]:
# create a BFMatcher object which will match up the SIFT features
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

matches = bf.match(dog1_desc, dog2_desc)

# Sort the matches in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)

# draw the top N matches
N_MATCHES = 100

match_img = cv2.drawMatches(
    dog1_img, dog1_kp,
    dog2_img, dog2_kp,
    matches[:N_MATCHES], dog2_img.copy(), flags=0)

plt.figure(figsize=(12,6))
plt.imshow(match_img);



In [5]:
import scipy.io
mat_Train = scipy.io.loadmat(my_data + '/train_list.mat')

In [6]:
mat_Train


{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct  9 08:36:13 2011',
 '__version__': '1.0',
 'annotation_list': array([[array(['n02085620-Chihuahua/n02085620_5927'], 
       dtype='<U34')],
        [array(['n02085620-Chihuahua/n02085620_4441'], 
       dtype='<U34')],
        [array(['n02085620-Chihuahua/n02085620_1502'], 
       dtype='<U34')],
        ..., 
        [ array(['n02116738-African_hunting_dog/n02116738_6754'], 
       dtype='<U44')],
        [ array(['n02116738-African_hunting_dog/n02116738_9333'], 
       dtype='<U44')],
        [ array(['n02116738-African_hunting_dog/n02116738_2503'], 
       dtype='<U44')]], dtype=object),
 'file_list': array([[array(['n02085620-Chihuahua/n02085620_5927.jpg'], 
       dtype='<U38')],
        [array(['n02085620-Chihuahua/n02085620_4441.jpg'], 
       dtype='<U38')],
        [array(['n02085620-Chihuahua/n02085620_1502.jpg'], 
       dtype='<U38')],
        ..., 
        [ array(['n02116738-A

In [45]:
dog_full_path = str(mat_Train['file_list'][11999][0][0])
dog_full_path

folder, file =  dog_full_path.split('/')

'n02116738-African_hunting_dog/n02116738_2503.jpg'

In [51]:
file

'n02116738_2503.jpg'

In [7]:
image_jpg_Train = np.apply_along_axis(lambda x : x[0][0].encode('UTF8'), 1, mat_Train['file_list'])

In [36]:
image_jpg_Train = np.apply_along_axis(lambda x : str(x[0][0]), 1, mat_Train['file_list'])

In [37]:
type(image_jpg_Train.tolist()[0])

str

In [38]:
image_jpg_Train.tolist()

['n02085620-Chihuahua/n02085620_5927.jpg',
 'n02085620-Chihuahua/n02085620_4441.jpg',
 'n02085620-Chihuahua/n02085620_1502.jpg',
 'n02085620-Chihuahua/n02085620_1916.jpg',
 'n02085620-Chihuahua/n02085620_13151.jp',
 'n02085620-Chihuahua/n02085620_1569.jpg',
 'n02085620-Chihuahua/n02085620_9654.jpg',
 'n02085620-Chihuahua/n02085620_3975.jpg',
 'n02085620-Chihuahua/n02085620_3942.jpg',
 'n02085620-Chihuahua/n02085620_9351.jpg',
 'n02085620-Chihuahua/n02085620_574.jpg',
 'n02085620-Chihuahua/n02085620_4998.jpg',
 'n02085620-Chihuahua/n02085620_7.jpg',
 'n02085620-Chihuahua/n02085620_1617.jpg',
 'n02085620-Chihuahua/n02085620_11140.jp',
 'n02085620-Chihuahua/n02085620_5771.jpg',
 'n02085620-Chihuahua/n02085620_275.jpg',
 'n02085620-Chihuahua/n02085620_9399.jpg',
 'n02085620-Chihuahua/n02085620_10621.jp',
 'n02085620-Chihuahua/n02085620_4266.jpg',
 'n02085620-Chihuahua/n02085620_7738.jpg',
 'n02085620-Chihuahua/n02085620_3110.jpg',
 'n02085620-Chihuahua/n02085620_12101.jp',
 'n02085620-Chih

In [9]:
image_jpg_Train

array([b'n02085620-Chihuahua/n02085620_5927.jpg',
       b'n02085620-Chihuahua/n02085620_4441.jpg',
       b'n02085620-Chihuahua/n02085620_1502.jpg', ...,
       b'n02116738-African_hunting_dog/n0211673',
       b'n02116738-African_hunting_dog/n0211673',
       b'n02116738-African_hunting_dog/n0211673'], 
      dtype='|S38')

In [10]:
mat_Train['file_list'][0]

array([array(['n02085620-Chihuahua/n02085620_5927.jpg'], 
      dtype='<U38')], dtype=object)

In [11]:
map(str, mat_Train['file_list'])

In [15]:
ascii_array = np.apply_along_axis(lambda y: [str(i) for i in y], 0, mat_Train['file_list']).tolist()

In [21]:
import re
re.match('\'(.*)\'', ascii_array[0])

TypeError: expected string or bytes-like object

In [28]:
my_df = pd.DataFrame(mat_Train['file_list'], columns=['file_path'])

In [25]:
image_jpg_Train


array([u'n02085620-Chihuahua/n02085620_5927.jpg',
       u'n02085620-Chihuahua/n02085620_4441.jpg',
       u'n02085620-Chihuahua/n02085620_1502.jpg', ...,
       u'n02116738-African_hunting_dog/n0211673',
       u'n02116738-African_hunting_dog/n0211673',
       u'n02116738-African_hunting_dog/n0211673'],
      dtype='<U38')

In [99]:
file_path = str(mat_Train['file_list'][0][0][0])

In [100]:
file_path

'n02085620-Chihuahua/n02085620_5927.jpg'

In [26]:

my_df = pd.DataFrame(image_jpg_Train, columns=['file_path'])

In [32]:
my_df['str_path'] = my_df['file_path'].astype(str)

In [33]:
my_df

,file_path,str_path
0,[n02085620-Chihuahua/n02085620_5927.jpg],[u'n02085620-Chihuahua/n02085620_5927.jpg']
1,[n02085620-Chihuahua/n02085620_4441.jpg],[u'n02085620-Chihuahua/n02085620_4441.jpg']
2,[n02085620-Chihuahua/n02085620_1502.jpg],[u'n02085620-Chihuahua/n02085620_1502.jpg']
3,[n02085620-Chihuahua/n02085620_1916.jpg],[u'n02085620-Chihuahua/n02085620_1916.jpg']
4,[n02085620-Chihuahua/n02085620_13151.jpg],[u'n02085620-Chihuahua/n02085620_13151.jpg']
5,[n02085620-Chihuahua/n02085620_1569.jpg],[u'n02085620-Chihuahua/n02085620_1569.jpg']
6,[n02085620-Chihuahua/n02085620_9654.jpg],[u'n02085620-Chihuahua/n02085620_9654.jpg']
7,[n02085620-Chihuahua/n02085620_3975.jpg],[u'n02085620-Chihuahua/n02085620_3975.jpg']
8,[n02085620-Chihuahua/n02085620_3942.jpg],[u'n02085620-Chihuahua/n02085620_3942.jpg']
9,[n02085620-Chihuahua/n02085620_9351.jpg],[u'n02085620-Chihuahua/n02085620_9351.jpg']


In [136]:
my_df['file_path'].astype(str).str.split('/')

0        [n02085620-Chihuahua, n02085620_5927.jpg]
1        [n02085620-Chihuahua, n02085620_4441.jpg]
2        [n02085620-Chihuahua, n02085620_1502.jpg]
3        [n02085620-Chihuahua, n02085620_1916.jpg]
4        [n02085620-Chihuahua, n02085620_13151.jp]
5        [n02085620-Chihuahua, n02085620_1569.jpg]
6        [n02085620-Chihuahua, n02085620_9654.jpg]
7        [n02085620-Chihuahua, n02085620_3975.jpg]
8        [n02085620-Chihuahua, n02085620_3942.jpg]
9        [n02085620-Chihuahua, n02085620_9351.jpg]
10        [n02085620-Chihuahua, n02085620_574.jpg]
11       [n02085620-Chihuahua, n02085620_4998.jpg]
12          [n02085620-Chihuahua, n02085620_7.jpg]
13       [n02085620-Chihuahua, n02085620_1617.jpg]
14       [n02085620-Chihuahua, n02085620_11140.jp]
15       [n02085620-Chihuahua, n02085620_5771.jpg]
16        [n02085620-Chihuahua, n02085620_275.jpg]
17       [n02085620-Chihuahua, n02085620_9399.jpg]
18       [n02085620-Chihuahua, n02085620_10621.jp]
19       [n02085620-Chihuahua, 

In [59]:
dogs_full_path = []
dogs_folder = []
dogs_file = []

for img_file in mat_Train['file_list']:
    dog_full_path = str(img_file[0][0])
    folder, file =  dog_full_path.split('/')
    dogs_full_path.append(dog_full_path)
    dogs_folder.append(folder)
    dogs_file.append(file)
    


In [80]:
pd.DataFrame({'file' : dogs_file,
             'folder' : dogs_folder,
             'full_path' : dogs_full_path})

,file,folder,full_path
0,n02085620_5927.jpg,n02085620-Chihuahua,n02085620-Chihuahua/n02085620_5927.jpg
1,n02085620_4441.jpg,n02085620-Chihuahua,n02085620-Chihuahua/n02085620_4441.jpg
2,n02085620_1502.jpg,n02085620-Chihuahua,n02085620-Chihuahua/n02085620_1502.jpg
3,n02085620_1916.jpg,n02085620-Chihuahua,n02085620-Chihuahua/n02085620_1916.jpg
4,n02085620_13151.jpg,n02085620-Chihuahua,n02085620-Chihuahua/n02085620_13151.jpg
5,n02085620_1569.jpg,n02085620-Chihuahua,n02085620-Chihuahua/n02085620_1569.jpg
6,n02085620_9654.jpg,n02085620-Chihuahua,n02085620-Chihuahua/n02085620_9654.jpg
7,n02085620_3975.jpg,n02085620-Chihuahua,n02085620-Chihuahua/n02085620_3975.jpg
8,n02085620_3942.jpg,n02085620-Chihuahua,n02085620-Chihuahua/n02085620_3942.jpg
9,n02085620_9351.jpg,n02085620-Chihuahua,n02085620-Chihuahua/n02085620_9351.jpg


In [77]:
pd.DataFrame(mat_Train['labels'], columns = ['label'])

,label
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1
